In [1]:
%reload_ext sparkmagic.magics
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

In [ ]:
## Spark config 
{"driverMemory": "2000M", "executorCores": 3,
 "conf": {"spark.ssl.enabled": false,"spark.jars": "local:///mounts/shared-volume/shared/postgresql-4260.jar","spark.sql.extensions":"io.delta.sql.DeltaSparkSessionExtension","spark.sql.catalog.spark_catalog":"org.apache.spark.sql.delta.catalog.DeltaCatalog" ,"spark.kubernetes.driver.volumes.persistentVolumeClaim.pv1.options.claimName": "kubeflow-shared-pvc", "spark.kubernetes.driver.volumes.persistentVolumeClaim.pv1.mount.path": "/mounts/shared-volume/shared", "spark.kubernetes.driver.volumes.persistentVolumeClaim.pv1.mount.readOnly": false, "spark.kubernetes.executor.volumes.persistentVolumeClaim.pv1.options.claimName": "kubeflow-shared-pvc", "spark.kubernetes.executor.volumes.persistentVolumeClaim.pv1.mount.path": "/mounts/shared-volume/shared", "spark.kubernetes.executor.volumes.persistentVolumeClaim.pv1.mount.readOnly": false, "spark.kubernetes.driver.volumes.persistentVolumeClaim.upv1.options.claimName": "hpadmin-spark-pvc", "spark.kubernetes.driver.volumes.persistentVolumeClaim.upv1.mount.path": "/mounts/shared-volume/user", "spark.kubernetes.driver.volumes.persistentVolumeClaim.upv1.mount.readOnly": false, "spark.kubernetes.executor.volumes.persistentVolumeClaim.upv1.options.claimName": "hpadmin-spark-pvc", "spark.kubernetes.executor.volumes.persistentVolumeClaim.upv1.mount.path": "/mounts/shared-volume/user", "spark.kubernetes.executor.volumes.persistentVolumeClaim.upv1.mount.readOnly": false, "spark.dynamicAllocation.enabled": true, "spark.dynamicAllocation.shuffleTracking.enabled": true, "spark.dynamicAllocation.minExecutors": 1, "spark.dynamicAllocation.maxExecutors": 4, "spark.kubernetes.container.image": "gcr.io/mapr-252711/spark-py-3.3.1:v3.3.1", "spark.kubernetes.container.image.pullPolicy": "Always", "spark.hadoop.fs.s3a.connection.ssl.enabled" : "false", "spark.hadoop.fs.s3a.endpoint" : "http://10.10.162.201:9000/", "spark.hadoop.fs.s3a.access.key" : "minioadmin","spark.hadoop.fs.s3a.secret.key" : "minioadmin","spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
"spark.driver.extraJavaOptions": "-Dcom.amazonaws.sdk.disableCertChecking=true","spark.executor.extraJavaOptions": "-Dcom.amazonaws.sdk.disableCertChecking=false"}}


In [16]:
## Livy credentials
#http://livy-0.livy-svc.spark.svc.cluster.local:8998
#username- hpadmin
#password- HP1nvent!

from pyspark.sql.functions import *
from delta.tables import *
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the raw parquet file of yellow taxi year 2021

In [32]:
df_yellow_2021=spark.read.parquet('file:///mounts/shared-volume/shared/dfm/silver/yellow/yellow_tripdata_2021-01.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_yellow_2021.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)

## Identify the tip amount which is greater than $1000 

In [ ]:
##get the subset of the dataframe. Fist create the view from dataframe
df_yellow_2021.createOrReplaceTempView("df_yellow_2021_view")
query = "SELECT * FROM df_yellow_2021_view WHERE tip_amount >1000"

In [ ]:
sub_set=spark.sql(query)

In [ ]:
sub_set.select('tip_amount').show()

## Write a table in delta lake format

In [34]:
df_yellow_2021.write.format("delta").save("file:///mounts/shared-volume/shared/dfm/silver/yellow/delta")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read data from delta lake tables

In [35]:
## Read data from delta lake tables
df_yellow_2021_delta = spark.read.format("delta").load("file:///mounts/shared-volume/shared/dfm/silver/yellow/delta")
df_yellow_2021_delta.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|            1.0|          2.1|       1.0|                 N|         142|          43|           2|        8.0|  3.0|    0.5|       0.

## update table data

### Overwrite

In [36]:
## read the parquet format of year 2020 (yellow taxi)
df_yellow_2020=spark.read.parquet('file:///mounts/shared-volume/shared/dfm/bronze/yellow/yellow_tripdata_2020-01.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
## transform the type of airport fee column to the double
df_yellow_2020=df_yellow_2020.withColumn("airport_fee",col("airport_fee").cast("double"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df_yellow_2020.write.format("delta").mode("overwrite").save("file:///mounts/shared-volume/shared/dfm/silver/yellow/delta")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Conditional overwrite merge aand update

In [39]:
deltaTable = DeltaTable.forPath(spark, "file:///mounts/shared-volume/shared/dfm/silver/yellow/delta")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("tip_amount == 1100"),
  set = { "tip_amount": expr("tip_amount - 100") })


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
deltaTable.show()

In [40]:
## Read the updated data
df_yellow_2021_update = spark.read.format("delta").load("file:///mounts/shared-volume/shared/dfm/silver/yellow/delta")
df_yellow_2021_update.createOrReplaceTempView("df_yellow_2021_update_view")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df_yellow_2021_update.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)

In [42]:
query = "SELECT * FROM df_yellow_2021_update_view WHERE tip_amount > 900"
sub_set=spark.sql(query)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
sub_set.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2020-01-22 16:41:24|  2020-01-22 16:49:28|            1.0|         0.55|       1.0|                 N|         231|         209|           1|        6.5|  1.0|    0.5|    1000.

In [15]:
sub_set.select('total_amount').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|total_amount|
+------------+
|      1110.8|
+------------+

In [44]:
from pyspark.sql.functions import col
# Define the condition for updating the column
condition = col("tip_amount") == 1000

# Define the new value you want to update the column with
new_value = 1

# Update the column using withColumn()
updated_df = sub_set.withColumn("VendorID", when(condition, new_value).otherwise(col("VendorID")))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
updated_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2020-01-22 16:41:24|  2020-01-22 16:49:28|            1.0|         0.55|       1.0|                 N|         231|         209|           1|        6.5|  1.0|    0.5|    1000.

In [46]:
deltaTable.alias("oldData") \
  .merge(
    updated_df.alias("newData"),
    "oldData.tip_amount = newData.tip_amount") \
  .whenMatchedUpdate(set = { "VendorID": col("newData.VendorID") }) \
  .whenNotMatchedInsert(values = { "VendorID": col("newData.VendorID") }) \
  .execute()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## History

In [47]:
fullHistoryDF = deltaTable.history() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
fullHistoryDF.select(col("timestamp"),col("operation"),col("version")).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+---------+-------+
|timestamp              |operation|version|
+-----------------------+---------+-------+
|2023-07-10 14:35:43.989|MERGE    |3      |
|2023-07-10 14:33:16.29 |UPDATE   |2      |
|2023-07-10 14:32:36.472|WRITE    |1      |
|2023-07-10 14:31:08.364|WRITE    |0      |
+-----------------------+---------+-------+

## Detail

In [49]:
detailDF = deltaTable.detail()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
detailDF.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------------------------+----+-----------+---------------------------------------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+
|format|id                                  |name|description|location                                                 |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|
+------+------------------------------------+----+-----------+---------------------------------------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+
|delta |9830b4bc-8ff6-4ae6-9821-a3eef3b3aa35|null|null       |file:/mounts/shared-volume/shared/dfm/silver/yellow/delta|2023-07-10 14:31:02.659|2023-07-10 14:35:43.989|[]              |5       |129122380  |{}        |1               |2         

In [31]:
## Save final dataframe to the postgres
# Assuming you have a DataFrame called "df" and want to save it to a PostgreSQL table

# Define the connection properties
jdbc_url = "jdbc:postgresql://10.10.162.202:5432/postgres"
table_name = "yellow"
connection_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

df=spark.read.format("delta").load("file:///mounts/shared-volume/shared/dfm/silver/yellow/delta")
# Save the DataFrame to PostgreSQL
df.write.jdbc(url=jdbc_url, table=table_name, mode="overwrite", properties=connection_properties)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…